In [1]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

# --- 0. Конфигурация ---

csv_file = 'Chicago_Traffic_Tracker_-_Historical_Congestion_Estimates_by_Segment_-_2024-Current.csv'
parquet_file = 'Chicago_Traffic_Tracker_-_Historical_Congestion_Estimates_by_Segment_-_2024-Current.parquet'
chunksize = 1_000_000
n_lines = 73_000_000

# Столбцы для удаления (COMMENTS часто бесполезны)
columns_to_drop = ['COMMENTS']

# Столбцы-кандидаты на преобразование в экономный тип 'category'
categorical_cols = ['DIRECTION', 'STREET_HEADING', 'STREET']

# Карта для оптимальных числовых типов
dtype_map = {
    # Маленькие целые числа -> uint8 (0-255)
    'HOUR': 'uint8',
    'DAY_OF_WEEK': 'uint8',
    'MONTH': 'uint8',
    # Стандартные целые числа -> int32
    'SEGMENT_ID': 'int32',
    'BUS_COUNT': 'int32',
    'MESSAGE_COUNT': 'int32',
    # Числа с плавающей точкой -> float32
    'SPEED': 'float32',
    'LENGTH': 'float32',
    'START_LATITUDE': 'float32',
    'START_LONGITUDE': 'float32',
    'END_LATITUDE': 'float32',
    'END_LONGITUDE': 'float32',
}

# Явно указываем столбец с датой и его формат
date_column = 'TIME'
date_format = '%m/%d/%Y %I:%M:%S %p'

In [2]:
tmp = pd.read_csv(csv_file, chunksize=100).__next__().head()
tmp

,TIME,SEGMENT_ID,SPEED,STREET,DIRECTION,FROM_STREET,TO_STREET,LENGTH,STREET_HEADING,COMMENTS,...,HOUR,DAY_OF_WEEK,MONTH,RECORD_ID,START_LATITUDE,START_LONGITUDE,END_LATITUDE,END_LONGITUDE,START_LOCATION,END_LOCATION
0,05/14/2025 12:30:49 PM,1,25,55th,EB,Pulaski,Central Park,0.5,W,NaN,...,12,4,5,0001-202505141730,41.793067,-87.723160,41.793141,-87.713607,POINT (-87.7231602513 41.7930671862),POINT (-87.7136071496 41.793140551)
1,05/14/2025 12:30:49 PM,2,27,55th,EB,Central Park,Kedzie,0.5,W,NaN,...,12,4,5,0002-202505141730,41.793141,-87.713607,41.793377,-87.703555,POINT (-87.7136071496 41.793140551),POINT (-87.7035554211 41.7933767009)
2,05/14/2025 12:30:49 PM,3,-1,55th,EB,Kedzie,California,0.5,W,NaN,...,12,4,5,0003-202505141730,41.793377,-87.703555,41.793554,-87.693799,POINT (-87.7035554211 41.7933767009),POINT (-87.693799302 41.7935536023)
3,05/14/2025 12:30:49 PM,4,-1,55th,EB,California,Western,0.5,W,NaN,...,12,4,5,0004-202505141730,41.793554,-87.693799,41.793739,-87.684043,POINT (-87.693799302 41.7935536023),POINT (-87.6840431555 41.7937393759)
4,05/14/2025 12:30:49 PM,5,23,Garfield,EB,Western,Damen,0.5,W,NaN,...,12,4,5,0005-202505141730,41.793739,-87.684043,41.793779,-87.674302,POINT (-87.6840431555 41.7937393759),POINT (-87.6743019717 41.7937791103)


In [3]:
# Столбцы для удаления (COMMENTS часто бесполезны)
columns_to_drop = ['COMMENTS']

# Столбцы-кандидаты на преобразование в экономный тип 'category'
categorical_cols = ['DIRECTION', 'STREET_HEADING', 'STREET']

# Карта для оптимальных числовых типов
dtype_map = {
    # Маленькие целые числа -> uint8 (0-255)
    'HOUR': 'uint8',
    'DAY_OF_WEEK': 'uint8',
    'MONTH': 'uint8',
    # Стандартные целые числа -> int32
    'SEGMENT_ID': 'int32',
    'BUS_COUNT': 'int32',
    'MESSAGE_COUNT': 'int32',
    # Числа с плавающей точкой -> float32
    'SPEED': 'float32',
    'LENGTH': 'float32',
    'START_LATITUDE': 'float32',
    'START_LONGITUDE': 'float32',
    'END_LATITUDE': 'float32',
    'END_LONGITUDE': 'float32',
}

# Явно указываем столбец с датой и его формат
date_column = 'TIME'
date_format = '%m/%d/%Y %I:%M:%S %p'

In [4]:
# --- 1. Первый проход: Сбор всех уникальных категорий ---

print("--- Начало первого прохода: Сбор уникальных категорий ---")
unique_values = {col: set() for col in categorical_cols}
pass1_iterator = pd.read_csv(csv_file, usecols=categorical_cols, chunksize=chunksize, low_memory=True)
for chunk in tqdm(pass1_iterator, total=n_lines // chunksize + 1):
    for col in categorical_cols:
        unique_values[col].update(chunk[col].dropna().unique())
print("Сбор уникальных значений завершен.")
for col, values in unique_values.items():
    print(f"  Найдено {len(values)} уникальных значений в столбце '{col}'.")
    
category_dtypes = {
    col: CategoricalDtype(categories=sorted(list(values)), ordered=False)
    for col, values in unique_values.items()
}

# --- 2. Второй проход: Конвертация и запись в Parquet ---

print("\n--- Начало второго прохода: Конвертация и запись данных ---")

def process_chunk(chunk, dtypes_map, cat_dtypes, date_col, date_fmt):
    """Функция для полной обработки одного чанка данных."""
    chunk.drop(columns=columns_to_drop, inplace=True, errors='ignore')

    for col, dtype in dtypes_map.items():
        if col in chunk.columns:
            chunk[col] = pd.to_numeric(chunk[col], errors='coerce').astype(dtype)

    for col, dtype in cat_dtypes.items():
        if col in chunk.columns:
            chunk[col] = chunk[col].astype(dtype)
            
    if date_col in chunk.columns:
        chunk[date_col] = pd.to_datetime(chunk[date_col], format=date_fmt, errors='coerce')
            
    return chunk

# Создаем итератор БЕЗ parse_dates. Столбец TIME будет прочитан как текст.
pass2_iterator = pd.read_csv(csv_file, iterator=True, chunksize=chunksize, low_memory=True)

# Обработка первого чанка для получения схемы
first_chunk = next(pass2_iterator)
first_chunk = process_chunk(first_chunk, dtype_map, category_dtypes, date_column, date_format)

# Получаем схему из обработанного чанка
schema = pa.Table.from_pandas(first_chunk, preserve_index=False).schema

# Запись в Parquet с помощью ParquetWriter
with pq.ParquetWriter(parquet_file, schema, compression='snappy') as writer:
    print("Обработана и записана часть 1...")
    writer.write_table(pa.Table.from_pandas(first_chunk, schema=schema, preserve_index=False))

    # Обрабатываем и записываем оставшиеся чанки
    for i, chunk in tqdm(enumerate(pass2_iterator, start=2), total=n_lines // chunksize):
        processed_chunk = process_chunk(chunk, dtype_map, category_dtypes, date_column, date_format)
        
        table = pa.Table.from_pandas(processed_chunk, schema=schema, preserve_index=False)
        writer.write_table(table)
        
        print(f"Обработана и записана часть {i}...")

print(f"\nКонвертация завершена! Оптимизированный файл '{parquet_file}' готов.")


--- Начало первого прохода: Сбор уникальных категорий ---


100%|██████████| 74/74 [02:08<00:00,  1.74s/it]


Сбор уникальных значений завершен.
  Найдено 8 уникальных значений в столбце 'DIRECTION'.
  Найдено 4 уникальных значений в столбце 'STREET_HEADING'.
  Найдено 62 уникальных значений в столбце 'STREET'.

--- Начало второго прохода: Конвертация и запись данных ---


/var/tmp/ipykernel_3678055/487410013.py:43: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  first_chunk = next(pass2_iterator)


Обработана и записана часть 1...


...

Обработана и записана часть 72...


 99%|█████████▊| 72/73 [07:57<00:04,  4.54s/it]

Обработана и записана часть 73...


100%|██████████| 73/73 [07:59<00:00,  6.57s/it]

Обработана и записана часть 74...

Конвертация завершена! Оптимизированный файл 'Chicago_Traffic_Tracker_-_Historical_Congestion_Estimates_by_Segment_-_2024-Current.parquet' готов.
